In [1]:
import otoole
print(otoole.__version__)
from otoole import read
import os
import sys
sys.path.append('../')

from otoole.utils import (
    _read_file,
)
import xarray as xr

import logging

from feo.osemosys.variables import add_variables
from feo.osemosys.model.constraints.demand import *
from feo.osemosys.model.constraints.capacity_adequacy_a import *
from feo.osemosys.model.constraints.capacity_adequacy_b import *
from feo.osemosys.model.constraints.energy_balance_a import *
from feo.osemosys.model.constraints.energy_balance_b import *
#from feo.osemosys.model.constraints.accounting_technology import *

logger = logging.getLogger(__name__)

1.1.2


In [2]:
model_name = 'model_one'
config_path = os.path.join('../tests/data', f'{model_name}.yaml')
folder_path = os.path.join('../tests/data', model_name)

print(config_path, folder_path)

../tests/data/model_one.yaml ../tests/data/model_one


In [3]:
with open(config_path, "r") as config_file:
    config = _read_file(config_file, '.yaml')

model, defaults = read(config_path, 'csv', folder_path)

# Reshape the CSV files to create the dataset of sets and parameters
# If the sparse package is available (Python <3.11)
# data_vars = {x: xr.DataArray().from_series(y.VALUE, sparse=True) for x, y in model.items() if config[x]['type'] == 'param'}
data_vars = {x: y.VALUE.to_xarray() for x, y in model.items() if config[x]['type'] == 'param'}
coords = {x: y.values.T[0] for x, y in model.items() if config[x]['type'] == 'set'}
ds = xr.Dataset(data_vars=data_vars, coords=coords)
ds = ds.assign_coords({'_REGION': model['REGION'].values.T[0]})

for param, default in defaults.items():
    if config[param]['type'] == 'param':
        ds[param].attrs['default'] = default
        if default != 0:
            ds[param] = ds[param].fillna(default)
# ds.to_netcdf(f'{model_name}.nc', engine='netcdf4')
# ds = xr.open_dataset(f'{model_name}.nc')
# ds = ds.drop_sel(YEAR=range(2025, 2071))

# Model Creation

In [4]:
# client = Client()
# client

In [5]:
from linopy import Model, solvers, available_solvers

chunks = {
    'YEAR': ds.YEAR.size,
    'TIMESLICE': ds.TIMESLICE.size,
    'REGION': 1,
    'TECHNOLOGY': 100,
    'FUEL': 100,
    'MODE_OF_OPERATION': ds.MODE_OF_OPERATION.size,
    'SEASON': ds.SEASON.size
}

m = Model(force_dim_names=True) #, chunk='auto')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


## Variables

In [6]:
m = add_variables(ds, m)

## Discounting

```ampl
param DiscountRate{r in REGION};
param DiscountRateIdv{r in REGION, t in TECHNOLOGY}, default DiscountRate[r];

param DiscountFactor{r in REGION, y in YEAR} :=
	(1 + DiscountRate[r]) ^ (y - min{yy in YEAR} min(yy) + 0.0);
param DiscountFactorMid{r in REGION, y in YEAR} :=
	(1 + DiscountRate[r]) ^ (y - min{yy in YEAR} min(yy) + 0.5);

param OperationalLife{r in REGION, t in TECHNOLOGY};

param CapitalRecoveryFactor{r in REGION, t in TECHNOLOGY} :=
	(1 - (1 + DiscountRateIdv[r,t])^(-1))/(1 - (1 + DiscountRateIdv[r,t])^(-(OperationalLife[r,t])));
param PvAnnuity{r in REGION, t in TECHNOLOGY} :=
	(1 - (1 + DiscountRate[r])^(-(OperationalLife[r,t]))) * (1 + DiscountRate[r]) / DiscountRate[r];

param DiscountRateStorage{r in REGION, s in STORAGE};
param DiscountFactorStorage{r in REGION, s in STORAGE, y in YEAR} :=
	(1 + DiscountRateStorage[r, s]) ^ (y - min{yy in YEAR} min(yy) + 0.0);
param DiscountFactorMidStorage{r in REGION, s in STORAGE, y in YEAR} :=
	(1 + DiscountRateStorage[r, s]) ^ (y - min{yy in YEAR} min(yy) + 0.5);
```

In [7]:
discount_factor = ((1 + ds['DiscountRate']) ** (ds.coords['YEAR'] - min(ds.coords['YEAR'])))
discount_factor_mid = ((1 + ds['DiscountRate']) ** (ds.coords['YEAR'] - min(ds.coords['YEAR']) + 0.5))

discount_factor_idv = ((1 + ds['DiscountRateIdv']) ** (ds.coords['YEAR'] - min(ds.coords['YEAR'])))
discount_factor_mid_idv = ((1 + ds['DiscountRateIdv']) ** (ds.coords['YEAR'] - min(ds.coords['YEAR']) + 0.5))

pv_annuity = (1 - (1 + ds['DiscountRateIdv'])**(-(ds['OperationalLife']))) * (1 + ds['DiscountRateIdv']) / ds['DiscountRateIdv']

capital_recovery_factor = (1 - (1 + ds['DiscountRateIdv'])**(-1))/(1 - (1 + ds['DiscountRateIdv'])**(-(ds['OperationalLife'])))

# Constraints

## Storage

```ampl
s.t. S1_RateOfStorageCharge{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}:
	sum{t in TECHNOLOGY, m in MODE_OF_OPERATION, l in TIMESLICE: TechnologyToStorage[r,t,s,m] > 0}
	RateOfActivity[r,l,t,m,y] * TechnologyToStorage[r,t,s,m] * Conversionls[l,ls] * Conversionld[l,ld] * Conversionlh[l,lh]
	=
	RateOfStorageCharge[r,s,ls,ld,lh,y];

s.t. S2_RateOfStorageDischarge{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}:
	sum{t in TECHNOLOGY, m in MODE_OF_OPERATION, l in TIMESLICE: TechnologyFromStorage[r,t,s,m] > 0}
	RateOfActivity[r,l,t,m,y] * TechnologyFromStorage[r,t,s,m] * Conversionls[l,ls] * Conversionld[l,ld] * Conversionlh[l,lh]
	=
	RateOfStorageDischarge[r,s,ls,ld,lh,y];

s.t. S3_NetChargeWithinYear{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}:
	sum{l in TIMESLICE:Conversionls[l,ls]>0 && Conversionld[l,ld] > 0 && Conversionlh[l,lh] > 0}
	(RateOfStorageCharge[r,s,ls,ld,lh,y] - RateOfStorageDischarge[r,s,ls,ld,lh,y]) * YearSplit[l,y] *
	Conversionls[l,ls] * Conversionld[l,ld] * Conversionlh[l,lh]
	=
	NetChargeWithinYear[r,s,ls,ld,lh,y];

s.t. S4_NetChargeWithinDay{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}:
	(RateOfStorageCharge[r,s,ls,ld,lh,y] - RateOfStorageDischarge[r,s,ls,ld,lh,y]) * DaySplit[lh,y]
	=
	NetChargeWithinDay[r,s,ls,ld,lh,y];

s.t. S5_and_S6_StorageLevelYearStart{r in REGION, s in STORAGE, y in YEAR}:
	if y = min{yy in YEAR} min(yy)
	then StorageLevelStart[r,s]
	else StorageLevelYearStart[r,s,y-1] + sum{ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET}
	NetChargeWithinYear[r,s,ls,ld,lh,y-1]
	=
	StorageLevelYearStart[r,s,y];

s.t. S7_and_S8_StorageLevelYearFinish{r in REGION, s in STORAGE, y in YEAR}:
	if y < max{yy in YEAR} max(yy)
	then StorageLevelYearStart[r,s,y+1]
	else StorageLevelYearStart[r,s,y] + sum{ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET}
	NetChargeWithinYear[r,s,ls,ld,lh,y]
	=
	StorageLevelYearFinish[r,s,y];

s.t. S9_and_S10_StorageLevelSeasonStart{r in REGION, s in STORAGE, ls in SEASON, y in YEAR}:
	if ls = min{lsls in SEASON} min(lsls)
	then StorageLevelYearStart[r,s,y]
	else StorageLevelSeasonStart[r,s,ls-1,y] + sum{ld in DAYTYPE, lh in DAILYTIMEBRACKET}
	NetChargeWithinYear[r,s,ls-1,ld,lh,y]
	=
	StorageLevelSeasonStart[r,s,ls,y];

s.t. S11_and_S12_StorageLevelDayTypeStart{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, y in YEAR}:
	if ld = min{ldld in DAYTYPE} min(ldld)
	then StorageLevelSeasonStart[r,s,ls,y]
	else StorageLevelDayTypeStart[r,s,ls,ld-1,y] + sum{lh in DAILYTIMEBRACKET}
	NetChargeWithinDay[r,s,ls,ld-1,lh,y] * DaysInDayType[ls,ld-1,y]
	=
	StorageLevelDayTypeStart[r,s,ls,ld,y];

s.t. S13_and_S14_and_S15_StorageLevelDayTypeFinish{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, y in YEAR}:
	if ls = max{lsls in SEASON} max(lsls) && ld = max{ldld in DAYTYPE} max(ldld)
	then StorageLevelYearFinish[r,s,y]
	else if ld = max{ldld in DAYTYPE} max(ldld)
	then StorageLevelSeasonStart[r,s,ls+1,y]
	else StorageLevelDayTypeFinish[r,s,ls,ld+1,y] - sum{lh in DAILYTIMEBRACKET}
	NetChargeWithinDay[r,s,ls,ld+1,lh,y] * DaysInDayType[ls,ld+1,y]
	=
	StorageLevelDayTypeFinish[r,s,ls,ld,y];

#
##########		Storage Constraints				#############
#
s.t. SC1_LowerLimit_BeginningOfDailyTimeBracketOfFirstInstanceOfDayTypeInFirstWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: 0 <= (StorageLevelDayTypeStart[r,s,ls,ld,y]+sum{lhlh in DAILYTIMEBRACKET:lh-lhlh>0} NetChargeWithinDay[r,s,ls,ld,lhlh,y])-StorageLowerLimit[r,s,y];
s.t. SC1_UpperLimit_BeginningOfDailyTimeBracketOfFirstInstanceOfDayTypeInFirstWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: (StorageLevelDayTypeStart[r,s,ls,ld,y]+sum{lhlh in DAILYTIMEBRACKET:lh-lhlh>0} NetChargeWithinDay[r,s,ls,ld,lhlh,y])-StorageUpperLimit[r,s,y] <= 0;
s.t. SC2_LowerLimit_EndOfDailyTimeBracketOfLastInstanceOfDayTypeInFirstWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: 0 <= if ld > min{ldld in DAYTYPE} min(ldld) then (StorageLevelDayTypeStart[r,s,ls,ld,y]-sum{lhlh in DAILYTIMEBRACKET:lh-lhlh<0} NetChargeWithinDay[r,s,ls,ld-1,lhlh,y])-StorageLowerLimit[r,s,y];
s.t. SC2_UpperLimit_EndOfDailyTimeBracketOfLastInstanceOfDayTypeInFirstWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: if ld > min{ldld in DAYTYPE} min(ldld) then (StorageLevelDayTypeStart[r,s,ls,ld,y]-sum{lhlh in DAILYTIMEBRACKET:lh-lhlh<0} NetChargeWithinDay[r,s,ls,ld-1,lhlh,y])-StorageUpperLimit[r,s,y] <= 0;
s.t. SC3_LowerLimit_EndOfDailyTimeBracketOfLastInstanceOfDayTypeInLastWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}:  0 <= (StorageLevelDayTypeFinish[r,s,ls,ld,y] - sum{lhlh in DAILYTIMEBRACKET:lh-lhlh<0} NetChargeWithinDay[r,s,ls,ld,lhlh,y])-StorageLowerLimit[r,s,y];
s.t. SC3_UpperLimit_EndOfDailyTimeBracketOfLastInstanceOfDayTypeInLastWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}:  (StorageLevelDayTypeFinish[r,s,ls,ld,y] - sum{lhlh in DAILYTIMEBRACKET:lh-lhlh<0} NetChargeWithinDay[r,s,ls,ld,lhlh,y])-StorageUpperLimit[r,s,y] <= 0;
s.t. SC4_LowerLimit_BeginningOfDailyTimeBracketOfFirstInstanceOfDayTypeInLastWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: 	0 <= if ld > min{ldld in DAYTYPE} min(ldld) then (StorageLevelDayTypeFinish[r,s,ls,ld-1,y]+sum{lhlh in DAILYTIMEBRACKET:lh-lhlh>0} NetChargeWithinDay[r,s,ls,ld,lhlh,y])-StorageLowerLimit[r,s,y];
s.t. SC4_UpperLimit_BeginningOfDailyTimeBracketOfFirstInstanceOfDayTypeInLastWeekConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: if ld > min{ldld in DAYTYPE} min(ldld) then (StorageLevelDayTypeFinish[r,s,ls,ld-1,y]+sum{lhlh in DAILYTIMEBRACKET:lh-lhlh>0} NetChargeWithinDay[r,s,ls,ld,lhlh,y])-StorageUpperLimit[r,s,y] <= 0;
s.t. SC5_MaxChargeConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: RateOfStorageCharge[r,s,ls,ld,lh,y] <= StorageMaxChargeRate[r,s];
s.t. SC6_MaxDischargeConstraint{r in REGION, s in STORAGE, ls in SEASON, ld in DAYTYPE, lh in DAILYTIMEBRACKET, y in YEAR}: RateOfStorageDischarge[r,s,ls,ld,lh,y] <= StorageMaxDischargeRate[r,s];
#
#########		Storage Investments				#############
#
s.t. SI1_StorageUpperLimit{r in REGION, s in STORAGE, y in YEAR}: AccumulatedNewStorageCapacity[r,s,y]+ResidualStorageCapacity[r,s,y] = StorageUpperLimit[r,s,y];
s.t. SI2_StorageLowerLimit{r in REGION, s in STORAGE, y in YEAR}: MinStorageCharge[r,s,y]*StorageUpperLimit[r,s,y] = StorageLowerLimit[r,s,y];
s.t. SI3_TotalNewStorage{r in REGION, s in STORAGE, y in YEAR}: sum{yy in YEAR: y-yy < OperationalLifeStorage[r,s] && y-yy>=0} NewStorageCapacity[r,s,yy]=AccumulatedNewStorageCapacity[r,s,y];
s.t. SI4_UndiscountedCapitalInvestmentStorage{r in REGION, s in STORAGE, y in YEAR}: CapitalCostStorage[r,s,y] * NewStorageCapacity[r,s,y] = CapitalInvestmentStorage[r,s,y];
s.t. SI5_DiscountingCapitalInvestmentStorage{r in REGION, s in STORAGE, y in YEAR}: CapitalInvestmentStorage[r,s,y]/(DiscountFactorStorage[r,s,y]) = DiscountedCapitalInvestmentStorage[r,s,y];
s.t. SI6_SalvageValueStorageAtEndOfPeriod1{r in REGION, s in STORAGE, y in YEAR: (y+OperationalLifeStorage[r,s]-1) <= (max{yy in YEAR} max(yy))}: 0 = SalvageValueStorage[r,s,y];
s.t. SI7_SalvageValueStorageAtEndOfPeriod2{r in REGION, s in STORAGE, y in YEAR: (DepreciationMethod[r]=1 && (y+OperationalLifeStorage[r,s]-1) > (max{yy in YEAR} max(yy)) && DiscountRateStorage[r,s]=0) || (DepreciationMethod[r]=2 && (y+OperationalLifeStorage[r,s]-1) > (max{yy in YEAR} max(yy)))}: CapitalInvestmentStorage[r,s,y]*(1-(max{yy in YEAR} max(yy) - y+1)/OperationalLifeStorage[r,s]) = SalvageValueStorage[r,s,y];
s.t. SI8_SalvageValueStorageAtEndOfPeriod3{r in REGION, s in STORAGE, y in YEAR: DepreciationMethod[r]=1 && (y+OperationalLifeStorage[r,s]-1) > (max{yy in YEAR} max(yy)) && DiscountRateStorage[r,s]>0}: CapitalInvestmentStorage[r,s,y]*(1-(((1+DiscountRateStorage[r,s])^(max{yy in YEAR} max(yy) - y+1)-1)/((1+DiscountRateStorage[r,s])^OperationalLifeStorage[r,s]-1))) = SalvageValueStorage[r,s,y];
s.t. SI9_SalvageValueStorageDiscountedToStartYear{r in REGION, s in STORAGE, y in YEAR}: SalvageValueStorage[r,s,y]/((1+DiscountRateStorage[r,s])^(max{yy in YEAR} max(yy)-min{yy in YEAR} min(yy)+1)) = DiscountedSalvageValueStorage[r,s,y];
s.t. SI10_TotalDiscountedCostByStorage{r in REGION, s in STORAGE, y in YEAR}: DiscountedCapitalInvestmentStorage[r,s,y]-DiscountedSalvageValueStorage[r,s,y] = TotalDiscountedStorageCost[r,s,y];
```

## Demand

In [8]:
m = add_demand_constraints(ds, m)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


## Capacity Adequacy A

In [9]:
m = add_capacity_adequacy_a_constraints(ds, m)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

## Capacity Adequacy B

In [10]:
m = add_capacity_adequacy_b_constraints(ds, m)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

## Energy Balance A

In [11]:
m = add_energy_balance_a_constraints(ds, m)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

In [12]:
print(m.constraints['EBa11_EnergyBalanceEachTS5'])

Constraint `EBa11_EnergyBalanceEachTS5` (REGION: 1, TIMESLICE: 8, FUEL: 3, YEAR: 51):
-------------------------------------------------------------------------------------
[R1, S1D1, WATER, 2020]: +1 Production[R1, S1D1, WATER, 2020] - 1 Demand[R1, S1D1, WATER, 2020] - 1 Use[R1, S1D1, WATER, 2020] - 0 Trade[R1, R1, S1D1, WATER, 2020]      ≥ -0.0
[R1, S1D1, WATER, 2021]: +1 Production[R1, S1D1, WATER, 2021] - 1 Demand[R1, S1D1, WATER, 2021] - 1 Use[R1, S1D1, WATER, 2021] - 0 Trade[R1, R1, S1D1, WATER, 2021]      ≥ -0.0
[R1, S1D1, WATER, 2022]: +1 Production[R1, S1D1, WATER, 2022] - 1 Demand[R1, S1D1, WATER, 2022] - 1 Use[R1, S1D1, WATER, 2022] - 0 Trade[R1, R1, S1D1, WATER, 2022]      ≥ -0.0
[R1, S1D1, WATER, 2023]: +1 Production[R1, S1D1, WATER, 2023] - 1 Demand[R1, S1D1, WATER, 2023] - 1 Use[R1, S1D1, WATER, 2023] - 0 Trade[R1, R1, S1D1, WATER, 2023]      ≥ -0.0
[R1, S1D1, WATER, 2024]: +1 Production[R1, S1D1, WATER, 2024] - 1 Demand[R1, S1D1, WATER, 2024] - 1 Use[R1, S1D1, WATER, 202

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


## Energy Balance B

In [13]:
m = add_energy_balance_b_constraints(ds, m)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

In [14]:
print(m.constraints)

linopy.model.Constraints
------------------------
 * EQ_SpecifiedDemand (REGION, TIMESLICE, FUEL, YEAR)
 * CAa1_TotalNewCapacity (REGION, TECHNOLOGY, YEAR)
 * CAa2_TotalAnnualCapacity (REGION, TECHNOLOGY, YEAR)
 * CAa3_TotalActivityOfEachTechnology (REGION, TIMESLICE, TECHNOLOGY, YEAR)
 * CAa4_Constraint_Capacity (REGION, TECHNOLOGY, TIMESLICE, YEAR)
 * CAa5_TotalNewCapacity (REGION, YEAR, TECHNOLOGY)
 * CAb1_PlannedMaintenance (YEAR, REGION, TECHNOLOGY)
 * EBa1_RateOfFuelProduction1 (REGION, FUEL, YEAR, TECHNOLOGY, MODE_OF_OPERATION, TIMESLICE)
 * EBa2_RateOfFuelProduction2 (REGION, TIMESLICE, TECHNOLOGY, FUEL, YEAR)
 * EBa3_RateOfFuelProduction3 (REGION, TIMESLICE, FUEL, YEAR)
 * EBa4_RateOfFuelUse1 (REGION, FUEL, YEAR, TECHNOLOGY, MODE_OF_OPERATION, TIMESLICE)
 * EBa5_RateOfFuelUse2 (REGION, TIMESLICE, TECHNOLOGY, FUEL, YEAR)
 * EBa6_RateOfFuelUse3 (REGION, TIMESLICE, FUEL, YEAR)
 * EBa7_EnergyBalanceEachTS1 (YEAR, TIMESLICE, REGION, FUEL)
 * EBa8_EnergyBalanceEachTS2 (YEAR, TIMESLI

In [15]:
print(m.constraints)

linopy.model.Constraints
------------------------
 * EQ_SpecifiedDemand (REGION, TIMESLICE, FUEL, YEAR)
 * CAa1_TotalNewCapacity (REGION, TECHNOLOGY, YEAR)
 * CAa2_TotalAnnualCapacity (REGION, TECHNOLOGY, YEAR)
 * CAa3_TotalActivityOfEachTechnology (REGION, TIMESLICE, TECHNOLOGY, YEAR)
 * CAa4_Constraint_Capacity (REGION, TECHNOLOGY, TIMESLICE, YEAR)
 * CAa5_TotalNewCapacity (REGION, YEAR, TECHNOLOGY)
 * CAb1_PlannedMaintenance (YEAR, REGION, TECHNOLOGY)
 * EBa1_RateOfFuelProduction1 (REGION, FUEL, YEAR, TECHNOLOGY, MODE_OF_OPERATION, TIMESLICE)
 * EBa2_RateOfFuelProduction2 (REGION, TIMESLICE, TECHNOLOGY, FUEL, YEAR)
 * EBa3_RateOfFuelProduction3 (REGION, TIMESLICE, FUEL, YEAR)
 * EBa4_RateOfFuelUse1 (REGION, FUEL, YEAR, TECHNOLOGY, MODE_OF_OPERATION, TIMESLICE)
 * EBa5_RateOfFuelUse2 (REGION, TIMESLICE, TECHNOLOGY, FUEL, YEAR)
 * EBa6_RateOfFuelUse3 (REGION, TIMESLICE, FUEL, YEAR)
 * EBa7_EnergyBalanceEachTS1 (YEAR, TIMESLICE, REGION, FUEL)
 * EBa8_EnergyBalanceEachTS2 (YEAR, TIMESLI

## Accounting Technology Production/Use

```ampl
s.t. Acc1_FuelProductionByTechnology{r in REGION, l in TIMESLICE, t in TECHNOLOGY, f in FUEL, y in YEAR}:
	RateOfProductionByTechnology[r,l,t,f,y] * YearSplit[l,y]
	=
	ProductionByTechnology[r,l,t,f,y];
```

In [16]:
con = (m['RateOfProductionByTechnology'] * ds['YearSplit']) - m['ProductionByTechnology'] == 0
mask = ds['OutputActivityRatio'].sum('MODE_OF_OPERATION') != 0
m.add_constraints(con, name='Acc1_FuelProductionByTechnology', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `Acc1_FuelProductionByTechnology` (YEAR: 51, TIMESLICE: 8, REGION: 1, TECHNOLOGY: 3, FUEL: 3):
---------------------------------------------------------------------------------------------------------
[2020, S1D1, R1, TRANSMISSION, WATER]: +0                                                                                                                                          = -0.0
[2020, S1D1, R1, TRANSMISSION, ELC_01]: +0                                                                                                                                         = -0.0
[2020, S1D1, R1, TRANSMISSION, ELC_02]: +0.125 RateOfProductionByTechnology[R1, S1D1, TRANSMISSION, ELC_02, 2020] - 1 ProductionByTechnology[R1, S1D1, TRANSMISSION, ELC_02, 2020] = -0.0
[2020, S1D1, R1, MINE_WATER, WATER]: +0.125 RateOfProductionByTechnology[R1, S1D1, MINE_WATER, WATER, 2020] - 1 ProductionByTechnology[R1, S1D1, MINE_WATER, WATER, 2020]          = -0.0
[2020, S1D1, R1, MINE_WATER, ELC_01]: +0    

```ampl
s.t. Acc2_FuelUseByTechnology{r in REGION, l in TIMESLICE, t in TECHNOLOGY, f in FUEL, y in YEAR}:
	RateOfUseByTechnology[r,l,t,f,y] * YearSplit[l,y]
	=
	UseByTechnology[r,l,t,f,y];
```

In [17]:
con = (m['RateOfUseByTechnology'] * ds['YearSplit']) - m['UseByTechnology'] == 0
mask = ds['InputActivityRatio'].sum('MODE_OF_OPERATION') != 0
m.add_constraints(con, name='Acc2_FuelUseByTechnology', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `Acc2_FuelUseByTechnology` (YEAR: 51, TIMESLICE: 8, REGION: 1, TECHNOLOGY: 3, FUEL: 3):
--------------------------------------------------------------------------------------------------
[2020, S1D1, R1, TRANSMISSION, WATER]: +0                                                                                                                            = -0.0
[2020, S1D1, R1, TRANSMISSION, ELC_01]: +0.125 RateOfUseByTechnology[R1, S1D1, TRANSMISSION, ELC_01, 2020] - 1 UseByTechnology[R1, S1D1, TRANSMISSION, ELC_01, 2020] = -0.0
[2020, S1D1, R1, TRANSMISSION, ELC_02]: +0                                                                                                                           = -0.0
[2020, S1D1, R1, MINE_WATER, WATER]: +0                                                                                                                              = -0.0
[2020, S1D1, R1, MINE_WATER, ELC_01]: +0                                                                          

```ampl
s.t. Acc3_AverageAnnualRateOfActivity{r in REGION, t in TECHNOLOGY, m in MODE_OF_OPERATION, y in YEAR}:
	sum{l in TIMESLICE} RateOfActivity[r,l,t,m,y]*YearSplit[l,y]
	=
	TotalAnnualTechnologyActivityByMode[r,t,m,y];
```

In [18]:
con = (m['RateOfActivity'] * ds['YearSplit']).sum('TIMESLICE') - m['TotalAnnualTechnologyActivityByMode'] == 0
# mask = ds['OutputActivityRatio'].sum('FUEL') != 0
m.add_constraints(con, name='Acc3_AverageAnnualRateOfActivity')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `Acc3_AverageAnnualRateOfActivity` (YEAR: 51, REGION: 1, TECHNOLOGY: 3, MODE_OF_OPERATION: 1):
---------------------------------------------------------------------------------------------------------
[2020, R1, TRANSMISSION, 1]: +0.125 RateOfActivity[R1, S1D1, TRANSMISSION, 1, 2020] + 0.125 RateOfActivity[R1, S1D2, TRANSMISSION, 1, 2020] + 0.125 RateOfActivity[R1, S2D1, TRANSMISSION, 1, 2020] ... +0.125 RateOfActivity[R1, S4D1, TRANSMISSION, 1, 2020] + 0.125 RateOfActivity[R1, S4D2, TRANSMISSION, 1, 2020] - 1 TotalAnnualTechnologyActivityByMode[R1, TRANSMISSION, 1, 2020] = -0.0
[2020, R1, MINE_WATER, 1]: +0.125 RateOfActivity[R1, S1D1, MINE_WATER, 1, 2020] + 0.125 RateOfActivity[R1, S1D2, MINE_WATER, 1, 2020] + 0.125 RateOfActivity[R1, S2D1, MINE_WATER, 1, 2020] ... +0.125 RateOfActivity[R1, S4D1, MINE_WATER, 1, 2020] + 0.125 RateOfActivity[R1, S4D2, MINE_WATER, 1, 2020] - 1 TotalAnnualTechnologyActivityByMode[R1, MINE_WATER, 1, 2020]               = -0.0
[2020, R1, HYDRO, 

```ampl
s.t. Acc4_ModelPeriodCostByRegion{r in REGION}:
	sum{y in YEAR}TotalDiscountedCost[r,y] = ModelPeriodCostByRegion[r];
```

In [19]:
con = m['TotalDiscountedCost'].sum('YEAR') - m['ModelPeriodCostByRegion'] == 0
m.add_constraints(con, name='Acc4_ModelPeriodCostByRegion')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `Acc4_ModelPeriodCostByRegion` (REGION: 1):
------------------------------------------------------
[R1]: +1 TotalDiscountedCost[R1, 2020] + 1 TotalDiscountedCost[R1, 2021] + 1 TotalDiscountedCost[R1, 2022] ... +1 TotalDiscountedCost[R1, 2069] + 1 TotalDiscountedCost[R1, 2070] - 1 ModelPeriodCostByRegion[R1] = -0.0

## Capital Costs

```ampl
s.t. CC1_UndiscountedCapitalInvestment{r in REGION, t in TECHNOLOGY, y in YEAR}: 
        CapitalCost[r,t,y] * NewCapacity[r,t,y] * CapitalRecoveryFactor[r,t] * PvAnnuity[r,t] 
        = 
        CapitalInvestment[r,t,y];
```

In [20]:
con = ds['CapitalCost'].fillna(0) * m['NewCapacity'] * capital_recovery_factor * pv_annuity - m['CapitalInvestment'] == 0
m.add_constraints(con, name='CC1_UndiscountedCapitalInvestment')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `CC1_UndiscountedCapitalInvestment` (REGION: 1, YEAR: 51, TECHNOLOGY: 3):
------------------------------------------------------------------------------------
[R1, 2020, TRANSMISSION]: +0 NewCapacity[R1, TRANSMISSION, 2020] - 1 CapitalInvestment[R1, TRANSMISSION, 2020] = -0.0
[R1, 2020, MINE_WATER]: +0 NewCapacity[R1, MINE_WATER, 2020] - 1 CapitalInvestment[R1, MINE_WATER, 2020]       = -0.0
[R1, 2020, HYDRO]: +4800 NewCapacity[R1, HYDRO, 2020] - 1 CapitalInvestment[R1, HYDRO, 2020]                   = -0.0
[R1, 2021, TRANSMISSION]: +0 NewCapacity[R1, TRANSMISSION, 2021] - 1 CapitalInvestment[R1, TRANSMISSION, 2021] = -0.0
[R1, 2021, MINE_WATER]: +0 NewCapacity[R1, MINE_WATER, 2021] - 1 CapitalInvestment[R1, MINE_WATER, 2021]       = -0.0
[R1, 2021, HYDRO]: +4800 NewCapacity[R1, HYDRO, 2021] - 1 CapitalInvestment[R1, HYDRO, 2021]                   = -0.0
[R1, 2022, TRANSMISSION]: +0 NewCapacity[R1, TRANSMISSION, 2022] - 1 CapitalInvestment[R1, TRANSMISSION, 2022] = -0.0
		..

```ampl
s.t. CC2_DiscountingCapitalInvestment{r in REGION, t in TECHNOLOGY, y in YEAR}: 
    CapitalInvestment[r,t,y]  / DiscountFactor[r,y] = DiscountedCapitalInvestment[r,t,y];
```

In [21]:
con = (m['CapitalInvestment'] / discount_factor) - m['DiscountedCapitalInvestment'] == 0
m.add_constraints(con, name='CC2_DiscountingCapitalInvestment')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `CC2_DiscountingCapitalInvestment` (REGION: 1, YEAR: 51, TECHNOLOGY: 3):
-----------------------------------------------------------------------------------
[R1, 2020, TRANSMISSION]: +1 CapitalInvestment[R1, TRANSMISSION, 2020] - 1 DiscountedCapitalInvestment[R1, TRANSMISSION, 2020]       = -0.0
[R1, 2020, MINE_WATER]: +1 CapitalInvestment[R1, MINE_WATER, 2020] - 1 DiscountedCapitalInvestment[R1, MINE_WATER, 2020]             = -0.0
[R1, 2020, HYDRO]: +1 CapitalInvestment[R1, HYDRO, 2020] - 1 DiscountedCapitalInvestment[R1, HYDRO, 2020]                            = -0.0
[R1, 2021, TRANSMISSION]: +0.9524 CapitalInvestment[R1, TRANSMISSION, 2021] - 1 DiscountedCapitalInvestment[R1, TRANSMISSION, 2021]  = -0.0
[R1, 2021, MINE_WATER]: +0.9524 CapitalInvestment[R1, MINE_WATER, 2021] - 1 DiscountedCapitalInvestment[R1, MINE_WATER, 2021]        = -0.0
[R1, 2021, HYDRO]: +0.9524 CapitalInvestment[R1, HYDRO, 2021] - 1 DiscountedCapitalInvestment[R1, HYDRO, 2021]                      

## Salvage Value

### GNU MathProg implementation
```ampl
s.t. SV1_SalvageValueAtEndOfPeriod1{
    r in REGION, t in TECHNOLOGY, y in YEAR: 
        DepreciationMethod[r]=1 && 
        (y + OperationalLife[r,t]-1) > (max{yy in YEAR} max(yy)) && 
        DiscountRate[r]>0}: 
    SalvageValue[r,t,y] 
    = 
    CapitalCost[r,t,y] * NewCapacity[r,t,y] * CapitalRecoveryFactor[r,t] * PvAnnuity[r,t] * 
    (1-(((1+DiscountRate[r])^(max{yy in YEAR} max(yy) - y+1)-1)/((1+DiscountRate[r])^OperationalLife[r,t]-1)));
```
### Pyomo implementation
```python
def SalvageValueAtEndOfPeriod1_rule(model, r, t, y):
    if (
        model.DepreciationMethod[r] == 1
        and ((y + model.OperationalLife[r, t] - 1) > max(model.YEAR))
        and model.DiscountRate[r] > 0
    ):
        return model.SalvageValue[r, t, y] == model.CapitalCost[
            r, t, y
        ] * model.NewCapacity[r, t, y] * (
            1
            - (
                ((1 + model.DiscountRate[r]) ** (max(model.YEAR) - y + 1) - 1)
                / ((1 + model.DiscountRate[r]) ** model.OperationalLife[r, t] - 1)
            )
        )
    elif (
        model.DepreciationMethod[r] == 1
        and ((y + model.OperationalLife[r, t] - 1) > max(model.YEAR))
        and model.DiscountRate[r] == 0
    ) or (
        model.DepreciationMethod[r] == 2
        and (y + model.OperationalLife[r, t] - 1) > (max(model.YEAR))
    ):
        return model.SalvageValue[r, t, y] == model.CapitalCost[
            r, t, y
        ] * model.NewCapacity[r, t, y] * (
            1 - (max(model.YEAR) - y + 1) / model.OperationalLife[r, t]
        )
    else:
        return model.SalvageValue[r, t, y] == 0


model.SalvageValueAtEndOfPeriod1 = Constraint(
    model.REGION, model.TECHNOLOGY, model.YEAR, rule=SalvageValueAtEndOfPeriod1_rule
)
```

In [22]:
def numerator(y: int):
    return ((1 + ds['DiscountRateIdv']) ** (max(ds.coords['YEAR']) - y + 1) - 1)

def denominator():
    return ((1 + ds['DiscountRateIdv']) ** ds['OperationalLife'] - 1)

def salvage_cost(ds):
    return ds['CapitalCost'].fillna(0) * (1 - (numerator(ds.coords['YEAR']) / denominator()))

con = m['SalvageValue'] - (m['NewCapacity'] * salvage_cost(ds)) == 0
mask = (ds['DepreciationMethod'] == 1) & ((ds.coords['YEAR'] + ds['OperationalLife'] - 1) > max(ds.coords['YEAR'])) & (ds['DiscountRateIdv'] > 0)
m.add_constraints(con, name='SV1_SalvageValueAtEndOfPeriod1', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `SV1_SalvageValueAtEndOfPeriod1` (REGION: 1, TECHNOLOGY: 3, YEAR: 51):
---------------------------------------------------------------------------------
[R1, TRANSMISSION, 2020]: +1 SalvageValue[R1, TRANSMISSION, 2020] + 0 NewCapacity[R1, TRANSMISSION, 2020] = -0.0
[R1, TRANSMISSION, 2021]: +1 SalvageValue[R1, TRANSMISSION, 2021] + 0 NewCapacity[R1, TRANSMISSION, 2021] = -0.0
[R1, TRANSMISSION, 2022]: +1 SalvageValue[R1, TRANSMISSION, 2022] + 0 NewCapacity[R1, TRANSMISSION, 2022] = -0.0
[R1, TRANSMISSION, 2023]: +1 SalvageValue[R1, TRANSMISSION, 2023] + 0 NewCapacity[R1, TRANSMISSION, 2023] = -0.0
[R1, TRANSMISSION, 2024]: +1 SalvageValue[R1, TRANSMISSION, 2024] + 0 NewCapacity[R1, TRANSMISSION, 2024] = -0.0
[R1, TRANSMISSION, 2025]: +1 SalvageValue[R1, TRANSMISSION, 2025] + 0 NewCapacity[R1, TRANSMISSION, 2025] = -0.0
[R1, TRANSMISSION, 2026]: +1 SalvageValue[R1, TRANSMISSION, 2026] + 0 NewCapacity[R1, TRANSMISSION, 2026] = -0.0
		...
[R1, HYDRO, 2064]: +1 SalvageValue[R1, 

```ampl
s.t. SV2_SalvageValueAtEndOfPeriod2{r in REGION, t in TECHNOLOGY, y in YEAR: 
        (DepreciationMethod[r]=1 && 
        (y + OperationalLife[r,t]-1) > (max{yy in YEAR} max(yy)) && 
        DiscountRate[r]=0) 
        || (DepreciationMethod[r]=2 && 
        (y + OperationalLife[r,t]-1) > (max{yy in YEAR} max(yy)))}: 
    SalvageValue[r,t,y] = CapitalCost[r,t,y] * NewCapacity[r,t,y] * CapitalRecoveryFactor[r,t] * PvAnnuity[r,t] *(1-(max{yy in YEAR} max(yy) - y+1)/OperationalLife[r,t]);
```

In [23]:
def numerator(y: int):
    return 1 - (max(ds.coords['YEAR']) - y + 1) - 1

def denominator():
    return ds['OperationalLife']

def salvage_cost(ds):
    return ds['CapitalCost'].fillna(0) * (1 - (numerator(ds.coords['YEAR']) / denominator()))

con = m['SalvageValue'] - (m['NewCapacity'] * salvage_cost(ds)) == 0
mask = ((ds['DepreciationMethod'] == 1) & ((ds.coords['YEAR'] + ds['OperationalLife'] - 1) > max(ds.coords['YEAR'])) & (ds['DiscountRateIdv'] == 0)) | ((ds['DepreciationMethod'] == 2) & ((ds.coords['YEAR'] + ds['OperationalLife'] - 1) > max(ds.coords['YEAR'])))
m.add_constraints(con, name='SV2_SalvageValueAtEndOfPeriod2', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `SV2_SalvageValueAtEndOfPeriod2` (REGION: 1, TECHNOLOGY: 3, YEAR: 51):
---------------------------------------------------------------------------------
[R1, TRANSMISSION, 2020]: +1 SalvageValue[R1, TRANSMISSION, 2020] - 0 NewCapacity[R1, TRANSMISSION, 2020] = -0.0
[R1, TRANSMISSION, 2021]: +1 SalvageValue[R1, TRANSMISSION, 2021] - 0 NewCapacity[R1, TRANSMISSION, 2021] = -0.0
[R1, TRANSMISSION, 2022]: +1 SalvageValue[R1, TRANSMISSION, 2022] - 0 NewCapacity[R1, TRANSMISSION, 2022] = -0.0
[R1, TRANSMISSION, 2023]: +1 SalvageValue[R1, TRANSMISSION, 2023] - 0 NewCapacity[R1, TRANSMISSION, 2023] = -0.0
[R1, TRANSMISSION, 2024]: +1 SalvageValue[R1, TRANSMISSION, 2024] - 0 NewCapacity[R1, TRANSMISSION, 2024] = -0.0
[R1, TRANSMISSION, 2025]: +1 SalvageValue[R1, TRANSMISSION, 2025] - 0 NewCapacity[R1, TRANSMISSION, 2025] = -0.0
[R1, TRANSMISSION, 2026]: +1 SalvageValue[R1, TRANSMISSION, 2026] - 0 NewCapacity[R1, TRANSMISSION, 2026] = -0.0
		...
[R1, HYDRO, 2064]: +1 SalvageValue[R1, 

```ampl
s.t. SV3_SalvageValueAtEndOfPeriod3{r in REGION, t in TECHNOLOGY, y in YEAR: (y + OperationalLife[r,t]-1) <= (max{yy in YEAR} max(yy))}: 
    SalvageValue[r,t,y] = 0;
```

In [24]:
con = m['SalvageValue'] == 0
mask = ((ds.coords['YEAR'] + ds['OperationalLife'] - 1) <= max(ds.coords['YEAR']))
m.add_constraints(con, name='SV3_SalvageValueAtEndOfPeriod3', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


Constraint `SV3_SalvageValueAtEndOfPeriod3` (REGION: 1, TECHNOLOGY: 3, YEAR: 51):
---------------------------------------------------------------------------------
[R1, TRANSMISSION, 2020]: +1 SalvageValue[R1, TRANSMISSION, 2020] = -0.0
[R1, TRANSMISSION, 2021]: +1 SalvageValue[R1, TRANSMISSION, 2021] = -0.0
[R1, TRANSMISSION, 2022]: +1 SalvageValue[R1, TRANSMISSION, 2022] = -0.0
[R1, TRANSMISSION, 2023]: +1 SalvageValue[R1, TRANSMISSION, 2023] = -0.0
[R1, TRANSMISSION, 2024]: +1 SalvageValue[R1, TRANSMISSION, 2024] = -0.0
[R1, TRANSMISSION, 2025]: +1 SalvageValue[R1, TRANSMISSION, 2025] = -0.0
[R1, TRANSMISSION, 2026]: +1 SalvageValue[R1, TRANSMISSION, 2026] = -0.0
		...
[R1, HYDRO, 2064]: +1 SalvageValue[R1, HYDRO, 2064]               = -0.0
[R1, HYDRO, 2065]: +1 SalvageValue[R1, HYDRO, 2065]               = -0.0
[R1, HYDRO, 2066]: +1 SalvageValue[R1, HYDRO, 2066]               = -0.0
[R1, HYDRO, 2067]: +1 SalvageValue[R1, HYDRO, 2067]               = -0.0
[R1, HYDRO, 2068]: +1 Salva

```ampl
s.t. SV4_SalvageValueDiscountedToStartYear{r in REGION, t in TECHNOLOGY, y in YEAR}: 
    DiscountedSalvageValue[r,t,y] = SalvageValue[r,t,y]/((1+DiscountRate[r])^(1+max{yy in YEAR} max(yy)-min{yy in YEAR} min(yy)));
```

In [25]:
def discounting(ds):
    return (1 + ds['DiscountRateIdv']) ** (1 + max(ds.coords['YEAR']) - min(ds.coords['YEAR']))
con = m['DiscountedSalvageValue'] - m['SalvageValue'] / discounting(ds) == 0
m.add_constraints(con, name='SV4_SalvageValueDiscountedToStartYear')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `SV4_SalvageValueDiscountedToStartYear` (REGION: 1, TECHNOLOGY: 3, YEAR: 51):
----------------------------------------------------------------------------------------
[R1, TRANSMISSION, 2020]: +1 DiscountedSalvageValue[R1, TRANSMISSION, 2020] - 0.08305 SalvageValue[R1, TRANSMISSION, 2020] = -0.0
[R1, TRANSMISSION, 2021]: +1 DiscountedSalvageValue[R1, TRANSMISSION, 2021] - 0.08305 SalvageValue[R1, TRANSMISSION, 2021] = -0.0
[R1, TRANSMISSION, 2022]: +1 DiscountedSalvageValue[R1, TRANSMISSION, 2022] - 0.08305 SalvageValue[R1, TRANSMISSION, 2022] = -0.0
[R1, TRANSMISSION, 2023]: +1 DiscountedSalvageValue[R1, TRANSMISSION, 2023] - 0.08305 SalvageValue[R1, TRANSMISSION, 2023] = -0.0
[R1, TRANSMISSION, 2024]: +1 DiscountedSalvageValue[R1, TRANSMISSION, 2024] - 0.08305 SalvageValue[R1, TRANSMISSION, 2024] = -0.0
[R1, TRANSMISSION, 2025]: +1 DiscountedSalvageValue[R1, TRANSMISSION, 2025] - 0.08305 SalvageValue[R1, TRANSMISSION, 2025] = -0.0
[R1, TRANSMISSION, 2026]: +1 DiscountedSal

## Operating Costs

```ampl
s.t. OC1_OperatingCostsVariable{r in REGION, t in TECHNOLOGY, y in YEAR: sum{m in MODE_OF_OPERATION} VariableCost[r,t,m,y] <> 0}:
	sum{m in MODE_OF_OPERATION}
	TotalAnnualTechnologyActivityByMode[r,t,m,y] * VariableCost[r,t,m,y]
	=
	AnnualVariableOperatingCost[r,t,y];
```

In [26]:
con = (m['TotalAnnualTechnologyActivityByMode'] * ds['VariableCost'].fillna(0)).sum(dims='MODE_OF_OPERATION') - m['AnnualVariableOperatingCost'] == 0
mask = (ds['VariableCost'].sum(dim='MODE_OF_OPERATION') != 0) & (~ds['VariableCost'].sum(dim='MODE_OF_OPERATION').isnull())
m.add_constraints(con, name='OC1_OperatingCostsVariable', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `OC1_OperatingCostsVariable` (REGION: 1, YEAR: 51, TECHNOLOGY: 3):
-----------------------------------------------------------------------------
[R1, 2020, TRANSMISSION]: +0 TotalAnnualTechnologyActivityByMode[R1, TRANSMISSION, 1, 2020] - 1 AnnualVariableOperatingCost[R1, TRANSMISSION, 2020] = -0.0
[R1, 2020, MINE_WATER]: +0 TotalAnnualTechnologyActivityByMode[R1, MINE_WATER, 1, 2020] - 1 AnnualVariableOperatingCost[R1, MINE_WATER, 2020]       = -0.0
[R1, 2020, HYDRO]: +0 TotalAnnualTechnologyActivityByMode[R1, HYDRO, 1, 2020] - 1 AnnualVariableOperatingCost[R1, HYDRO, 2020]                      = -0.0
[R1, 2021, TRANSMISSION]: +0 TotalAnnualTechnologyActivityByMode[R1, TRANSMISSION, 1, 2021] - 1 AnnualVariableOperatingCost[R1, TRANSMISSION, 2021] = -0.0
[R1, 2021, MINE_WATER]: +0 TotalAnnualTechnologyActivityByMode[R1, MINE_WATER, 1, 2021] - 1 AnnualVariableOperatingCost[R1, MINE_WATER, 2021]       = -0.0
[R1, 2021, HYDRO]: +0 TotalAnnualTechnologyActivityByMode[R1, HYDRO, 

```ampl
s.t. OC2_OperatingCostsFixedAnnual{r in REGION, t in TECHNOLOGY, y in YEAR}:
	TotalCapacityAnnual[r,t,y]*FixedCost[r,t,y]
	=
	AnnualFixedOperatingCost[r,t,y];
```

In [27]:
con = (m['TotalCapacityAnnual'] * ds['FixedCost'].fillna(0)) - m['AnnualFixedOperatingCost'] == 0
# mask = ~ds['FixedCost'].isnull()
m.add_constraints(con, name='OC2_OperatingCostsFixedAnnual')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `OC2_OperatingCostsFixedAnnual` (REGION: 1, YEAR: 51, TECHNOLOGY: 3):
--------------------------------------------------------------------------------
[R1, 2020, TRANSMISSION]: +0 TotalCapacityAnnual[R1, TRANSMISSION, 2020] - 1 AnnualFixedOperatingCost[R1, TRANSMISSION, 2020] = -0.0
[R1, 2020, MINE_WATER]: +0 TotalCapacityAnnual[R1, MINE_WATER, 2020] - 1 AnnualFixedOperatingCost[R1, MINE_WATER, 2020]       = -0.0
[R1, 2020, HYDRO]: +100 TotalCapacityAnnual[R1, HYDRO, 2020] - 1 AnnualFixedOperatingCost[R1, HYDRO, 2020]                    = -0.0
[R1, 2021, TRANSMISSION]: +0 TotalCapacityAnnual[R1, TRANSMISSION, 2021] - 1 AnnualFixedOperatingCost[R1, TRANSMISSION, 2021] = -0.0
[R1, 2021, MINE_WATER]: +0 TotalCapacityAnnual[R1, MINE_WATER, 2021] - 1 AnnualFixedOperatingCost[R1, MINE_WATER, 2021]       = -0.0
[R1, 2021, HYDRO]: +100 TotalCapacityAnnual[R1, HYDRO, 2021] - 1 AnnualFixedOperatingCost[R1, HYDRO, 2021]                    = -0.0
[R1, 2022, TRANSMISSION]: +0 TotalCapaci

```ampl
s.t. OC3_OperatingCostsTotalAnnual{r in REGION, t in TECHNOLOGY, y in YEAR}:
	AnnualFixedOperatingCost[r,t,y] + AnnualVariableOperatingCost[r,t,y]
	=
	OperatingCost[r,t,y];
```

In [28]:
con = m['AnnualFixedOperatingCost'] + m['AnnualVariableOperatingCost'] - m['OperatingCost'] == 0
# mask = (ds['VariableCost'].sum(dim='MODE_OF_OPERATION') != 0) & (~ds['FixedCost'].isnull())
m.add_constraints(con, name='OC3_OperatingCostsTotalAnnual')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `OC3_OperatingCostsTotalAnnual` (REGION: 1, TECHNOLOGY: 3, YEAR: 51):
--------------------------------------------------------------------------------
[R1, TRANSMISSION, 2020]: +1 AnnualFixedOperatingCost[R1, TRANSMISSION, 2020] + 1 AnnualVariableOperatingCost[R1, TRANSMISSION, 2020] - 1 OperatingCost[R1, TRANSMISSION, 2020] = -0.0
[R1, TRANSMISSION, 2021]: +1 AnnualFixedOperatingCost[R1, TRANSMISSION, 2021] + 1 AnnualVariableOperatingCost[R1, TRANSMISSION, 2021] - 1 OperatingCost[R1, TRANSMISSION, 2021] = -0.0
[R1, TRANSMISSION, 2022]: +1 AnnualFixedOperatingCost[R1, TRANSMISSION, 2022] + 1 AnnualVariableOperatingCost[R1, TRANSMISSION, 2022] - 1 OperatingCost[R1, TRANSMISSION, 2022] = -0.0
[R1, TRANSMISSION, 2023]: +1 AnnualFixedOperatingCost[R1, TRANSMISSION, 2023] + 1 AnnualVariableOperatingCost[R1, TRANSMISSION, 2023] - 1 OperatingCost[R1, TRANSMISSION, 2023] = -0.0
[R1, TRANSMISSION, 2024]: +1 AnnualFixedOperatingCost[R1, TRANSMISSION, 2024] + 1 AnnualVariableOperatingC

```ampl
s.t. OC4_DiscountedOperatingCostsTotalAnnual{r in REGION, t in TECHNOLOGY, y in YEAR}:
	OperatingCost[r,t,y] / DiscountFactorMid[r, y]
	=
	DiscountedOperatingCost[r,t,y];
```

In [29]:
con = m['OperatingCost'] / discount_factor_mid - m['DiscountedOperatingCost'] == 0
m.add_constraints(con, name='OC4_DiscountedOperatingCostsTotalAnnual')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `OC4_DiscountedOperatingCostsTotalAnnual` (REGION: 1, YEAR: 51, TECHNOLOGY: 3):
------------------------------------------------------------------------------------------
[R1, 2020, TRANSMISSION]: +0.9759 OperatingCost[R1, TRANSMISSION, 2020] - 1 DiscountedOperatingCost[R1, TRANSMISSION, 2020]  = -0.0
[R1, 2020, MINE_WATER]: +0.9759 OperatingCost[R1, MINE_WATER, 2020] - 1 DiscountedOperatingCost[R1, MINE_WATER, 2020]        = -0.0
[R1, 2020, HYDRO]: +0.9759 OperatingCost[R1, HYDRO, 2020] - 1 DiscountedOperatingCost[R1, HYDRO, 2020]                       = -0.0
[R1, 2021, TRANSMISSION]: +0.9294 OperatingCost[R1, TRANSMISSION, 2021] - 1 DiscountedOperatingCost[R1, TRANSMISSION, 2021]  = -0.0
[R1, 2021, MINE_WATER]: +0.9294 OperatingCost[R1, MINE_WATER, 2021] - 1 DiscountedOperatingCost[R1, MINE_WATER, 2021]        = -0.0
[R1, 2021, HYDRO]: +0.9294 OperatingCost[R1, HYDRO, 2021] - 1 DiscountedOperatingCost[R1, HYDRO, 2021]                       = -0.0
[R1, 2022, TRANSMISSION]: 

## Total Discounted Costs

```ampl
s.t. TDC1_TotalDiscountedCostByTechnology{r in REGION, t in TECHNOLOGY, y in YEAR}: 
    DiscountedOperatingCost[r,t,y] + DiscountedCapitalInvestment[r,t,y] 
    + DiscountedTechnologyEmissionsPenalty[r,t,y] - DiscountedSalvageValue[r,t,y] = TotalDiscountedCostByTechnology[r,t,y];
```

In [30]:
con = m['DiscountedOperatingCost'] + m['DiscountedCapitalInvestment'] + m['DiscountedTechnologyEmissionsPenalty'] \
        - m['DiscountedSalvageValue'] - m['TotalDiscountedCostByTechnology'] == 0
m.add_constraints(con, name='TDC1_TotalDiscountedCostByTechnology')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `TDC1_TotalDiscountedCostByTechnology` (REGION: 1, TECHNOLOGY: 3, YEAR: 51):
---------------------------------------------------------------------------------------
[R1, TRANSMISSION, 2020]: +1 DiscountedOperatingCost[R1, TRANSMISSION, 2020] + 1 DiscountedCapitalInvestment[R1, TRANSMISSION, 2020] + 1 DiscountedTechnologyEmissionsPenalty[R1, TRANSMISSION, 2020] - 1 DiscountedSalvageValue[R1, TRANSMISSION, 2020] - 1 TotalDiscountedCostByTechnology[R1, TRANSMISSION, 2020] = -0.0
[R1, TRANSMISSION, 2021]: +1 DiscountedOperatingCost[R1, TRANSMISSION, 2021] + 1 DiscountedCapitalInvestment[R1, TRANSMISSION, 2021] + 1 DiscountedTechnologyEmissionsPenalty[R1, TRANSMISSION, 2021] - 1 DiscountedSalvageValue[R1, TRANSMISSION, 2021] - 1 TotalDiscountedCostByTechnology[R1, TRANSMISSION, 2021] = -0.0
[R1, TRANSMISSION, 2022]: +1 DiscountedOperatingCost[R1, TRANSMISSION, 2022] + 1 DiscountedCapitalInvestment[R1, TRANSMISSION, 2022] + 1 DiscountedTechnologyEmissionsPenalty[R1, TRANSMISSION, 

```ampl
s.t. TDC2_TotalDiscountedCost{r in REGION, y in YEAR}: 
    sum{t in TECHNOLOGY} TotalDiscountedCostByTechnology[r,t,y] + 
    sum{s in STORAGE} TotalDiscountedStorageCost[r,s,y] 
    = TotalDiscountedCost[r,y];
```

In [31]:
try:
    con = m['TotalDiscountedCostByTechnology'].sum('TECHNOLOGY') + m['TotalDiscountedStorageCost'].sum('STORAGE') - m['TotalDiscountedCost'] == 0
except KeyError as ex:
    con = m['TotalDiscountedCostByTechnology'].sum('TECHNOLOGY') - m['TotalDiscountedCost'] == 0
finally:
    m.add_constraints(con, name='TDC2_TotalDiscountedCost')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

## Total Capacity Constraints

```ampl
s.t. TCC1_TotalAnnualMaxCapacityConstraint{r in REGION, t in TECHNOLOGY, y in YEAR: TotalAnnualMaxCapacity[r,t,y] <> -1}: 
    TotalCapacityAnnual[r,t,y] <= TotalAnnualMaxCapacity[r,t,y];
```

In [32]:
con = m['TotalCapacityAnnual'] <= ds['TotalAnnualMaxCapacity']
mask = ds['TotalAnnualMaxCapacity'] >= 0
m.add_constraints(con, name='TCC1_TotalAnnualMaxCapacityConstraint', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `TCC1_TotalAnnualMaxCapacityConstraint` (REGION: 1, TECHNOLOGY: 3, YEAR: 51):
----------------------------------------------------------------------------------------
[R1, TRANSMISSION, 2020]: +1 TotalCapacityAnnual[R1, TRANSMISSION, 2020] ≤ -1.0
[R1, TRANSMISSION, 2021]: +1 TotalCapacityAnnual[R1, TRANSMISSION, 2021] ≤ -1.0
[R1, TRANSMISSION, 2022]: +1 TotalCapacityAnnual[R1, TRANSMISSION, 2022] ≤ -1.0
[R1, TRANSMISSION, 2023]: +1 TotalCapacityAnnual[R1, TRANSMISSION, 2023] ≤ -1.0
[R1, TRANSMISSION, 2024]: +1 TotalCapacityAnnual[R1, TRANSMISSION, 2024] ≤ -1.0
[R1, TRANSMISSION, 2025]: +1 TotalCapacityAnnual[R1, TRANSMISSION, 2025] ≤ -1.0
[R1, TRANSMISSION, 2026]: +1 TotalCapacityAnnual[R1, TRANSMISSION, 2026] ≤ -1.0
		...
[R1, HYDRO, 2064]: +1 TotalCapacityAnnual[R1, HYDRO, 2064]               ≤ -1.0
[R1, HYDRO, 2065]: +1 TotalCapacityAnnual[R1, HYDRO, 2065]               ≤ -1.0
[R1, HYDRO, 2066]: +1 TotalCapacityAnnual[R1, HYDRO, 2066]               ≤ -1.0
[R1, HYDRO, 2067

```ampl
s.t. TCC2_TotalAnnualMinCapacityConstraint{r in REGION, t in TECHNOLOGY, y in YEAR: TotalAnnualMinCapacity[r,t,y]>0}: 
    TotalCapacityAnnual[r,t,y] >= TotalAnnualMinCapacity[r,t,y];
```

In [33]:
con = m['TotalCapacityAnnual'] >= ds['TotalAnnualMinCapacity']
mask = ds['TotalAnnualMinCapacity'] > 0
m.add_constraints(con, name='TCC2_TotalAnnualMinCapacityConstraint', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


Constraint `TCC2_TotalAnnualMinCapacityConstraint` (REGION: 1, TECHNOLOGY: 3, YEAR: 51):
----------------------------------------------------------------------------------------
[R1, TRANSMISSION, 2020]: +1 TotalCapacityAnnual[R1, TRANSMISSION, 2020] ≥ -0.0
[R1, TRANSMISSION, 2021]: +1 TotalCapacityAnnual[R1, TRANSMISSION, 2021] ≥ -0.0
[R1, TRANSMISSION, 2022]: +1 TotalCapacityAnnual[R1, TRANSMISSION, 2022] ≥ -0.0
[R1, TRANSMISSION, 2023]: +1 TotalCapacityAnnual[R1, TRANSMISSION, 2023] ≥ -0.0
[R1, TRANSMISSION, 2024]: +1 TotalCapacityAnnual[R1, TRANSMISSION, 2024] ≥ -0.0
[R1, TRANSMISSION, 2025]: +1 TotalCapacityAnnual[R1, TRANSMISSION, 2025] ≥ -0.0
[R1, TRANSMISSION, 2026]: +1 TotalCapacityAnnual[R1, TRANSMISSION, 2026] ≥ -0.0
		...
[R1, HYDRO, 2064]: +1 TotalCapacityAnnual[R1, HYDRO, 2064]               ≥ -0.0
[R1, HYDRO, 2065]: +1 TotalCapacityAnnual[R1, HYDRO, 2065]               ≥ -0.0
[R1, HYDRO, 2066]: +1 TotalCapacityAnnual[R1, HYDRO, 2066]               ≥ -0.0
[R1, HYDRO, 2067

## Annual Activity Constraints

```ampl
s.t. AAC1_TotalAnnualTechnologyActivity{r in REGION, t in TECHNOLOGY, y in YEAR}: 
    sum{l in TIMESLICE} RateOfTotalActivity[r,t,l,y]*YearSplit[l,y] = TotalTechnologyAnnualActivity[r,t,y];
```



In [34]:
con = (m['RateOfTotalActivity'] * ds['YearSplit']).sum('TIMESLICE') - m['TotalTechnologyAnnualActivity'] == 0
m.add_constraints(con, name='AAC1_TotalAnnualTechnologyActivity')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `AAC1_TotalAnnualTechnologyActivity` (YEAR: 51, REGION: 1, TECHNOLOGY: 3):
-------------------------------------------------------------------------------------
[2020, R1, TRANSMISSION]: +0.125 RateOfTotalActivity[R1, TRANSMISSION, S1D1, 2020] + 0.125 RateOfTotalActivity[R1, TRANSMISSION, S1D2, 2020] + 0.125 RateOfTotalActivity[R1, TRANSMISSION, S2D1, 2020] ... +0.125 RateOfTotalActivity[R1, TRANSMISSION, S4D1, 2020] + 0.125 RateOfTotalActivity[R1, TRANSMISSION, S4D2, 2020] - 1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2020] = -0.0
[2020, R1, MINE_WATER]: +0.125 RateOfTotalActivity[R1, MINE_WATER, S1D1, 2020] + 0.125 RateOfTotalActivity[R1, MINE_WATER, S1D2, 2020] + 0.125 RateOfTotalActivity[R1, MINE_WATER, S2D1, 2020] ... +0.125 RateOfTotalActivity[R1, MINE_WATER, S4D1, 2020] + 0.125 RateOfTotalActivity[R1, MINE_WATER, S4D2, 2020] - 1 TotalTechnologyAnnualActivity[R1, MINE_WATER, 2020]               = -0.0
[2020, R1, HYDRO]: +0.125 RateOfTotalActivity[R1, HYDRO, S1D1,

```ampl
s.t. AAC2_TotalAnnualTechnologyActivityUpperLimit{r in REGION, t in TECHNOLOGY, y in YEAR: TotalTechnologyAnnualActivityUpperLimit[r,t,y] <> -1}:
    TotalTechnologyAnnualActivity[r,t,y] <= TotalTechnologyAnnualActivityUpperLimit[r,t,y] ;
```

In [35]:
con = m['TotalTechnologyAnnualActivity'] <= ds['TotalTechnologyAnnualActivityUpperLimit']
mask = ds['TotalTechnologyAnnualActivityUpperLimit'] >= 0
m.add_constraints(con, name='AAC2_TotalAnnualTechnologyActivityUpperLimit', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


Constraint `AAC2_TotalAnnualTechnologyActivityUpperLimit` (REGION: 1, TECHNOLOGY: 3, YEAR: 51):
-----------------------------------------------------------------------------------------------
[R1, TRANSMISSION, 2020]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2020] ≤ -1.0
[R1, TRANSMISSION, 2021]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2021] ≤ -1.0
[R1, TRANSMISSION, 2022]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2022] ≤ -1.0
[R1, TRANSMISSION, 2023]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2023] ≤ -1.0
[R1, TRANSMISSION, 2024]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2024] ≤ -1.0
[R1, TRANSMISSION, 2025]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2025] ≤ -1.0
[R1, TRANSMISSION, 2026]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2026] ≤ -1.0
		...
[R1, HYDRO, 2064]: +1 TotalTechnologyAnnualActivity[R1, HYDRO, 2064]               ≤ -1.0
[R1, HYDRO, 2065]: +1 TotalTechnologyAnnualActivity[R1, HYDRO, 2065]              

```ampl
s.t. AAC3_TotalAnnualTechnologyActivityLowerLimit{r in REGION, t in TECHNOLOGY, y in YEAR: TotalTechnologyAnnualActivityLowerLimit[r,t,y]>0}: 
    TotalTechnologyAnnualActivity[r,t,y] >= TotalTechnologyAnnualActivityLowerLimit[r,t,y] ;
```

In [36]:
con = m['TotalTechnologyAnnualActivity'] >= ds['TotalTechnologyAnnualActivityLowerLimit']
mask = ds['TotalTechnologyAnnualActivityLowerLimit'] > 0
m.add_constraints(con, name='AAC3_TotalAnnualTechnologyActivityLowerLimit', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


Constraint `AAC3_TotalAnnualTechnologyActivityLowerLimit` (REGION: 1, TECHNOLOGY: 3, YEAR: 51):
-----------------------------------------------------------------------------------------------
[R1, TRANSMISSION, 2020]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2020] ≥ -0.0
[R1, TRANSMISSION, 2021]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2021] ≥ -0.0
[R1, TRANSMISSION, 2022]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2022] ≥ -0.0
[R1, TRANSMISSION, 2023]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2023] ≥ -0.0
[R1, TRANSMISSION, 2024]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2024] ≥ -0.0
[R1, TRANSMISSION, 2025]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2025] ≥ -0.0
[R1, TRANSMISSION, 2026]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2026] ≥ -0.0
		...
[R1, HYDRO, 2064]: +1 TotalTechnologyAnnualActivity[R1, HYDRO, 2064]               ≥ -0.0
[R1, HYDRO, 2065]: +1 TotalTechnologyAnnualActivity[R1, HYDRO, 2065]              

## Total Activity Constraints

```ampl
s.t. TAC1_TotalModelHorizonTechnologyActivity{r in REGION, t in TECHNOLOGY}: 
    sum{y in YEAR} TotalTechnologyAnnualActivity[r,t,y] = TotalTechnologyModelPeriodActivity[r,t];
```

In [37]:
con = m['TotalTechnologyAnnualActivity'].sum('YEAR') - m['TotalTechnologyModelPeriodActivity'] == 0
m.add_constraints(con, name='TAC1_TotalModelHorizonTechnologyActivity')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `TAC1_TotalModelHorizonTechnologyActivity` (REGION: 1, TECHNOLOGY: 3):
---------------------------------------------------------------------------------
[R1, TRANSMISSION]: +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2020] + 1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2021] + 1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2022] ... +1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2069] + 1 TotalTechnologyAnnualActivity[R1, TRANSMISSION, 2070] - 1 TotalTechnologyModelPeriodActivity[R1, TRANSMISSION] = -0.0
[R1, MINE_WATER]: +1 TotalTechnologyAnnualActivity[R1, MINE_WATER, 2020] + 1 TotalTechnologyAnnualActivity[R1, MINE_WATER, 2021] + 1 TotalTechnologyAnnualActivity[R1, MINE_WATER, 2022] ... +1 TotalTechnologyAnnualActivity[R1, MINE_WATER, 2069] + 1 TotalTechnologyAnnualActivity[R1, MINE_WATER, 2070] - 1 TotalTechnologyModelPeriodActivity[R1, MINE_WATER]               = -0.0
[R1, HYDRO]: +1 TotalTechnologyAnnualActivity[R1, HYDRO, 2020] + 1 TotalTechnologyAn

```ampl
s.t. TAC2_TotalModelHorizonTechnologyActivityUpperLimit{r in REGION, t in TECHNOLOGY: TotalTechnologyModelPeriodActivityUpperLimit[r,t]<>-1}: 
    TotalTechnologyModelPeriodActivity[r,t] <= TotalTechnologyModelPeriodActivityUpperLimit[r,t] ;
```

In [38]:
con = m['TotalTechnologyModelPeriodActivity'] <= ds['TotalTechnologyModelPeriodActivityUpperLimit']
mask = ds['TotalTechnologyModelPeriodActivityUpperLimit'] >= 0
m.add_constraints(con, name='TAC2_TotalModelHorizonTechnologyActivityUpperLimit', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


Constraint `TAC2_TotalModelHorizonTechnologyActivityUpperLimit` (REGION: 1, TECHNOLOGY: 3):
-------------------------------------------------------------------------------------------
[R1, TRANSMISSION]: +1 TotalTechnologyModelPeriodActivity[R1, TRANSMISSION] ≤ -1.0
[R1, MINE_WATER]: +1 TotalTechnologyModelPeriodActivity[R1, MINE_WATER]     ≤ -1.0
[R1, HYDRO]: +1 TotalTechnologyModelPeriodActivity[R1, HYDRO]               ≤ -1.0

```ampl
s.t. TAC3_TotalModelHorizenTechnologyActivityLowerLimit{r in REGION, t in TECHNOLOGY: TotalTechnologyModelPeriodActivityLowerLimit[r,t]>0}: 
    TotalTechnologyModelPeriodActivity[r,t] >= TotalTechnologyModelPeriodActivityLowerLimit[r,t] ;
```

In [39]:
con = m['TotalTechnologyModelPeriodActivity'] >= ds['TotalTechnologyModelPeriodActivityLowerLimit']
mask = ds['TotalTechnologyModelPeriodActivityLowerLimit'] > 0
m.add_constraints(con, name='TAC3_TotalModelHorizenTechnologyActivityLowerLimit', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


Constraint `TAC3_TotalModelHorizenTechnologyActivityLowerLimit` (REGION: 1, TECHNOLOGY: 3):
-------------------------------------------------------------------------------------------
[R1, TRANSMISSION]: +1 TotalTechnologyModelPeriodActivity[R1, TRANSMISSION] ≥ -0.0
[R1, MINE_WATER]: +1 TotalTechnologyModelPeriodActivity[R1, MINE_WATER]     ≥ -0.0
[R1, HYDRO]: +1 TotalTechnologyModelPeriodActivity[R1, HYDRO]               ≥ -0.0

## Reserve Margin Constraint

```ampl
s.t. RM1_ReserveMargin_TechnologiesIncluded_In_Activity_Units{r in REGION, l in TIMESLICE, y in YEAR: ReserveMargin[r,y] > 0}:
	sum {t in TECHNOLOGY}
	TotalCapacityAnnual[r,t,y] * ReserveMarginTagTechnology[r,t,y] * CapacityToActivityUnit[r,t]
	=
	TotalCapacityInReserveMargin[r,y];
```

In [40]:
con = ds['ReserveMarginTagTechnology'] * ds['CapacityToActivityUnit'] * m['TotalCapacityAnnual'] - m['TotalCapacityInReserveMargin'] == 0
mask = (ds['ReserveMargin'] > 0) & (ds['ReserveMarginTagTechnology'] == 1)
m.add_constraints(con, name='RM1_ReserveMargin_TechnologiesIncluded_In_Activity_Units', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `RM1_ReserveMargin_TechnologiesIncluded_In_Activity_Units` (REGION: 1, YEAR: 51, TECHNOLOGY: 3):
-----------------------------------------------------------------------------------------------------------
[R1, 2020, TRANSMISSION]: +nan TotalCapacityAnnual[R1, TRANSMISSION, 2020] - 1 TotalCapacityInReserveMargin[R1, 2020] = -0.0
[R1, 2020, MINE_WATER]: +nan TotalCapacityAnnual[R1, MINE_WATER, 2020] - 1 TotalCapacityInReserveMargin[R1, 2020]     = -0.0
[R1, 2020, HYDRO]: +nan TotalCapacityAnnual[R1, HYDRO, 2020] - 1 TotalCapacityInReserveMargin[R1, 2020]               = -0.0
[R1, 2021, TRANSMISSION]: +nan TotalCapacityAnnual[R1, TRANSMISSION, 2021] - 1 TotalCapacityInReserveMargin[R1, 2021] = -0.0
[R1, 2021, MINE_WATER]: +nan TotalCapacityAnnual[R1, MINE_WATER, 2021] - 1 TotalCapacityInReserveMargin[R1, 2021]     = -0.0
[R1, 2021, HYDRO]: +nan TotalCapacityAnnual[R1, HYDRO, 2021] - 1 TotalCapacityInReserveMargin[R1, 2021]               = -0.0
[R1, 2022, TRANSMISSION]: +nan Tot

```amlp
s.t. RM2_ReserveMargin_FuelsIncluded{r in REGION, l in TIMESLICE, y in YEAR: ReserveMargin[r,y] > 0}:
	sum {f in FUEL}
	RateOfProduction[r,l,f,y] * ReserveMarginTagFuel[r,f,y]
	=
	DemandNeedingReserveMargin[r,l,y];
```


In [41]:
con = ds['ReserveMarginTagFuel'] * m['RateOfProduction'] - m['DemandNeedingReserveMargin'] == 0
mask = (ds['ReserveMargin'] > 0) & (ds['ReserveMarginTagFuel'] == 1)
m.add_constraints(con, name='RM2_ReserveMargin_FuelsIncluded', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `RM2_ReserveMargin_FuelsIncluded` (REGION: 1, FUEL: 3, YEAR: 51, TIMESLICE: 8):
------------------------------------------------------------------------------------------
[R1, WATER, 2020, S1D1]: +nan RateOfProduction[R1, S1D1, WATER, 2020] - 1 DemandNeedingReserveMargin[R1, S1D1, 2020]   = -0.0
[R1, WATER, 2020, S1D2]: +nan RateOfProduction[R1, S1D2, WATER, 2020] - 1 DemandNeedingReserveMargin[R1, S1D2, 2020]   = -0.0
[R1, WATER, 2020, S2D1]: +nan RateOfProduction[R1, S2D1, WATER, 2020] - 1 DemandNeedingReserveMargin[R1, S2D1, 2020]   = -0.0
[R1, WATER, 2020, S2D2]: +nan RateOfProduction[R1, S2D2, WATER, 2020] - 1 DemandNeedingReserveMargin[R1, S2D2, 2020]   = -0.0
[R1, WATER, 2020, S3D1]: +nan RateOfProduction[R1, S3D1, WATER, 2020] - 1 DemandNeedingReserveMargin[R1, S3D1, 2020]   = -0.0
[R1, WATER, 2020, S3D2]: +nan RateOfProduction[R1, S3D2, WATER, 2020] - 1 DemandNeedingReserveMargin[R1, S3D2, 2020]   = -0.0
[R1, WATER, 2020, S4D1]: +nan RateOfProduction[R1, S4D1, WATER

```amlp
s.t. RM3_ReserveMargin_Constraint{r in REGION, l in TIMESLICE, y in YEAR: ReserveMargin[r,y] > 0}:
	DemandNeedingReserveMargin[r,l,y] * ReserveMargin[r,y]
	<=
	TotalCapacityInReserveMargin[r,y];
	```

In [42]:
con = ds['ReserveMargin'] * m['DemandNeedingReserveMargin'] - m['TotalCapacityInReserveMargin'] == 0
mask = ds['ReserveMargin'] > 0
m.add_constraints(con, name='RM3_ReserveMargin_Constraint', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `RM3_ReserveMargin_Constraint` (REGION: 1, YEAR: 51, TIMESLICE: 8):
------------------------------------------------------------------------------
[R1, 2020, S1D1]: +1 DemandNeedingReserveMargin[R1, S1D1, 2020] - 1 TotalCapacityInReserveMargin[R1, 2020] = -0.0
[R1, 2020, S1D2]: +1 DemandNeedingReserveMargin[R1, S1D2, 2020] - 1 TotalCapacityInReserveMargin[R1, 2020] = -0.0
[R1, 2020, S2D1]: +1 DemandNeedingReserveMargin[R1, S2D1, 2020] - 1 TotalCapacityInReserveMargin[R1, 2020] = -0.0
[R1, 2020, S2D2]: +1 DemandNeedingReserveMargin[R1, S2D2, 2020] - 1 TotalCapacityInReserveMargin[R1, 2020] = -0.0
[R1, 2020, S3D1]: +1 DemandNeedingReserveMargin[R1, S3D1, 2020] - 1 TotalCapacityInReserveMargin[R1, 2020] = -0.0
[R1, 2020, S3D2]: +1 DemandNeedingReserveMargin[R1, S3D2, 2020] - 1 TotalCapacityInReserveMargin[R1, 2020] = -0.0
[R1, 2020, S4D1]: +1 DemandNeedingReserveMargin[R1, S4D1, 2020] - 1 TotalCapacityInReserveMargin[R1, 2020] = -0.0
		...
[R1, 2070, S1D2]: +1 DemandNeedingRese

# RE Production Target NTS: Should change demand for production
```ampl
s.t. RE1_FuelProductionByTechnologyAnnual{r in REGION, t in TECHNOLOGY, f in FUEL, y in YEAR}: 
    sum{l in TIMESLICE} ProductionByTechnology[r,l,t,f,y] = ProductionByTechnologyAnnual[r,t,f,y];
```

In [43]:
con = m['ProductionByTechnology'].sum('TIMESLICE') - m['ProductionByTechnologyAnnual'] == 0
mask = ds['OutputActivityRatio'].sum('MODE_OF_OPERATION') != 0
m.add_constraints(con, name='RE1_FuelProductionByTechnologyAnnual', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `RE1_FuelProductionByTechnologyAnnual` (REGION: 1, TECHNOLOGY: 3, FUEL: 3, YEAR: 51):
------------------------------------------------------------------------------------------------
[R1, TRANSMISSION, WATER, 2020]: +0 = -0.0
[R1, TRANSMISSION, WATER, 2021]: +0 = -0.0
[R1, TRANSMISSION, WATER, 2022]: +0 = -0.0
[R1, TRANSMISSION, WATER, 2023]: +0 = -0.0
[R1, TRANSMISSION, WATER, 2024]: +0 = -0.0
[R1, TRANSMISSION, WATER, 2025]: +0 = -0.0
[R1, TRANSMISSION, WATER, 2026]: +0 = -0.0
		...
[R1, HYDRO, ELC_02, 2064]: +0       = -0.0
[R1, HYDRO, ELC_02, 2065]: +0       = -0.0
[R1, HYDRO, ELC_02, 2066]: +0       = -0.0
[R1, HYDRO, ELC_02, 2067]: +0       = -0.0
[R1, HYDRO, ELC_02, 2068]: +0       = -0.0
[R1, HYDRO, ELC_02, 2069]: +0       = -0.0
[R1, HYDRO, ELC_02, 2070]: +0       = -0.0

```ampl
s.t. RE2_TechIncluded{r in REGION, y in YEAR}: sum{t in TECHNOLOGY, f in FUEL} ProductionByTechnologyAnnual[r,t,f,y]*RETagTechnology[r,t,y] = TotalREProductionAnnual[r,y];
```

In [44]:
con = m['ProductionByTechnologyAnnual'] * ds['RETagTechnology'].fillna(0) - m['TotalREProductionAnnual'] == 0
mask = ds['RETagTechnology'] == 1
m.add_constraints(con, name='RE2_TechIncluded', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `RE2_TechIncluded` (REGION: 1, YEAR: 51, TECHNOLOGY: 3, FUEL: 3):
----------------------------------------------------------------------------
[R1, 2020, TRANSMISSION, WATER]: -1 TotalREProductionAnnual[R1, 2020]                                                                   = -0.0
[R1, 2020, TRANSMISSION, ELC_01]: -1 TotalREProductionAnnual[R1, 2020]                                                                  = -0.0
[R1, 2020, TRANSMISSION, ELC_02]: +0 ProductionByTechnologyAnnual[R1, TRANSMISSION, ELC_02, 2020] - 1 TotalREProductionAnnual[R1, 2020] = -0.0
[R1, 2020, MINE_WATER, WATER]: +0 ProductionByTechnologyAnnual[R1, MINE_WATER, WATER, 2020] - 1 TotalREProductionAnnual[R1, 2020]       = -0.0
[R1, 2020, MINE_WATER, ELC_01]: -1 TotalREProductionAnnual[R1, 2020]                                                                    = -0.0
[R1, 2020, MINE_WATER, ELC_02]: -1 TotalREProductionAnnual[R1, 2020]                                                               

```ampl
s.t. RE3_FuelIncluded{r in REGION, y in YEAR}: sum{l in TIMESLICE, f in FUEL} 
    RateOfProduction[r,l,f,y]*YearSplit[l,y]*RETagFuel[r,f,y] = RETotalProductionOfTargetFuelAnnual[r,y];
```

In [45]:
con = m['RateOfProduction'] * ds['YearSplit'] * ds['RETagFuel'].fillna(0) - m['RETotalProductionOfTargetFuelAnnual'] == 0
mask = ds['RETagFuel'] == 1
m.add_constraints(con, name='RE3_FuelIncluded', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `RE3_FuelIncluded` (YEAR: 51, TIMESLICE: 8, REGION: 1, FUEL: 3):
---------------------------------------------------------------------------
[2020, S1D1, R1, WATER]: +0 RateOfProduction[R1, S1D1, WATER, 2020] - 1 RETotalProductionOfTargetFuelAnnual[R1, 2020]   = -0.0
[2020, S1D1, R1, ELC_01]: +0 RateOfProduction[R1, S1D1, ELC_01, 2020] - 1 RETotalProductionOfTargetFuelAnnual[R1, 2020] = -0.0
[2020, S1D1, R1, ELC_02]: +0 RateOfProduction[R1, S1D1, ELC_02, 2020] - 1 RETotalProductionOfTargetFuelAnnual[R1, 2020] = -0.0
[2020, S1D2, R1, WATER]: +0 RateOfProduction[R1, S1D2, WATER, 2020] - 1 RETotalProductionOfTargetFuelAnnual[R1, 2020]   = -0.0
[2020, S1D2, R1, ELC_01]: +0 RateOfProduction[R1, S1D2, ELC_01, 2020] - 1 RETotalProductionOfTargetFuelAnnual[R1, 2020] = -0.0
[2020, S1D2, R1, ELC_02]: +0 RateOfProduction[R1, S1D2, ELC_02, 2020] - 1 RETotalProductionOfTargetFuelAnnual[R1, 2020] = -0.0
[2020, S2D1, R1, WATER]: +0 RateOfProduction[R1, S2D1, WATER, 2020] - 1 RETotalProduct

```ampl
s.t. RE4_EnergyConstraint{r in REGION, y in YEAR}:
    REMinProductionTarget[r,y] * RETotalProductionOfTargetFuelAnnual[r,y] <= TotalREProductionAnnual[r,y];
```

In [46]:
con = m['RETotalProductionOfTargetFuelAnnual'] * ds['REMinProductionTarget'].fillna(0) - m['TotalREProductionAnnual'] <= 0
mask = ds['REMinProductionTarget'] > 0
m.add_constraints(con, name='RE4_EnergyConstraint', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `RE4_EnergyConstraint` (REGION: 1, YEAR: 51):
--------------------------------------------------------
[R1, 2020]: +0 RETotalProductionOfTargetFuelAnnual[R1, 2020] - 1 TotalREProductionAnnual[R1, 2020] ≤ -0.0
[R1, 2021]: +0 RETotalProductionOfTargetFuelAnnual[R1, 2021] - 1 TotalREProductionAnnual[R1, 2021] ≤ -0.0
[R1, 2022]: +0 RETotalProductionOfTargetFuelAnnual[R1, 2022] - 1 TotalREProductionAnnual[R1, 2022] ≤ -0.0
[R1, 2023]: +0 RETotalProductionOfTargetFuelAnnual[R1, 2023] - 1 TotalREProductionAnnual[R1, 2023] ≤ -0.0
[R1, 2024]: +0 RETotalProductionOfTargetFuelAnnual[R1, 2024] - 1 TotalREProductionAnnual[R1, 2024] ≤ -0.0
[R1, 2025]: +0 RETotalProductionOfTargetFuelAnnual[R1, 2025] - 1 TotalREProductionAnnual[R1, 2025] ≤ -0.0
[R1, 2026]: +0 RETotalProductionOfTargetFuelAnnual[R1, 2026] - 1 TotalREProductionAnnual[R1, 2026] ≤ -0.0
		...
[R1, 2064]: +0 RETotalProductionOfTargetFuelAnnual[R1, 2064] - 1 TotalREProductionAnnual[R1, 2064] ≤ -0.0
[R1, 2065]: +0 RETotalProduction

```ampl
s.t. RE5_FuelUseByTechnologyAnnual{r in REGION, t in TECHNOLOGY, f in FUEL, y in YEAR}: 
    sum{l in TIMESLICE} RateOfUseByTechnology[r,l,t,f,y] * YearSplit[l,y] = UseByTechnologyAnnual[r,t,f,y];
```

In [47]:
con = (ds['YearSplit'] * m['RateOfUseByTechnology']).sum('TIMESLICE') - m['UseByTechnologyAnnual'] == 0
mask = ds['InputActivityRatio'].sum('MODE_OF_OPERATION') > 0
m.add_constraints(con, name='RE5_FuelUseByTechnologyAnnual', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `RE5_FuelUseByTechnologyAnnual` (YEAR: 51, REGION: 1, TECHNOLOGY: 3, FUEL: 3):
-----------------------------------------------------------------------------------------
[2020, R1, TRANSMISSION, WATER]: +0                                                                                                                                                                                                                                                                                                                                                                                                               = -0.0
[2020, R1, TRANSMISSION, ELC_01]: +0.125 RateOfUseByTechnology[R1, S1D1, TRANSMISSION, ELC_01, 2020] + 0.125 RateOfUseByTechnology[R1, S1D2, TRANSMISSION, ELC_01, 2020] + 0.125 RateOfUseByTechnology[R1, S2D1, TRANSMISSION, ELC_01, 2020] ... +0.125 RateOfUseByTechnology[R1, S4D1, TRANSMISSION, ELC_01, 2020] + 0.125 RateOfUseByTechnology[R1, S4D2, TRANSMISSION, ELC_01, 2020] - 1

## Emissions

```ampl
s.t. E1_AnnualEmissionProductionByMode{r in REGION, t in TECHNOLOGY, e in EMISSION, m in MODE_OF_OPERATION, y in YEAR:
									   EmissionActivityRatio[r,t,e,m,y] <> 0}:
	EmissionActivityRatio[r,t,e,m,y] * TotalAnnualTechnologyActivityByMode[r,t,m,y]
	=
	AnnualTechnologyEmissionByMode[r,t,e,m,y];
```


In [48]:
mask = ds['EmissionActivityRatio'].notnull()
con = ds['EmissionActivityRatio'] * m['TotalAnnualTechnologyActivityByMode'] - m['AnnualTechnologyEmissionByMode'] == 0
m.add_constraints(con, name='E1_AnnualEmissionProductionByMode', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `E1_AnnualEmissionProductionByMode` (REGION: 1, YEAR: 51, EMISSION: 0, TECHNOLOGY: 3, MODE_OF_OPERATION: 1):
-----------------------------------------------------------------------------------------------------------------------

```ampl
s.t. E2_AnnualEmissionProduction{r in REGION, t in TECHNOLOGY, e in EMISSION, y in YEAR}:
	sum{m in MODE_OF_OPERATION}
	AnnualTechnologyEmissionByMode[r,t,e,m,y]
	=
	AnnualTechnologyEmission[r,t,e,y];
```

In [49]:
if ds['EMISSION'].size > 0:
    mask = ds['EmissionActivityRatio'].sum('MODE_OF_OPERATION') != 0
    con = m['AnnualTechnologyEmissionByMode'].sum(dims='MODE_OF_OPERATION') - m['AnnualTechnologyEmission'] == 0
    m.add_constraints(con, name='E2_AnnualEmissionProduction', mask=mask)

```ampl
s.t. E3_EmissionsPenaltyByTechAndEmission{r in REGION, t in TECHNOLOGY, e in EMISSION, y in YEAR: EmissionsPenalty[r,e,y] <> 0}:
	AnnualTechnologyEmission[r,t,e,y] * EmissionsPenalty[r,e,y]
	=
	AnnualTechnologyEmissionPenaltyByEmission[r,t,e,y];
```

In [50]:
mask = (ds['EmissionsPenalty'].notnull()) & (ds['EmissionActivityRatio'].sum('MODE_OF_OPERATION') != 0)
con = (m['AnnualTechnologyEmission'] * ds['EmissionsPenalty']) - m['AnnualTechnologyEmissionPenaltyByEmission'] == 0
m.add_constraints(con, name='E3_EmissionsPenaltyByTechAndEmission', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `E3_EmissionsPenaltyByTechAndEmission` (REGION: 1, YEAR: 51, EMISSION: 0, TECHNOLOGY: 3):
----------------------------------------------------------------------------------------------------

```ampl
s.t. E4_EmissionsPenaltyByTechnology{r in REGION, t in TECHNOLOGY, y in YEAR}:
	sum{e in EMISSION} AnnualTechnologyEmissionPenaltyByEmission[r,t,e,y]
	=
	AnnualTechnologyEmissionsPenalty[r,t,y];
```


In [51]:
con = m['AnnualTechnologyEmissionPenaltyByEmission'].sum('EMISSION') - m['AnnualTechnologyEmissionsPenalty'] == 0
m.add_constraints(con, name='E4_EmissionsPenaltyByTechnology')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `E4_EmissionsPenaltyByTechnology` (REGION: 1, TECHNOLOGY: 3, YEAR: 51):
----------------------------------------------------------------------------------
[R1, TRANSMISSION, 2020]: -1 AnnualTechnologyEmissionsPenalty[R1, TRANSMISSION, 2020] = -0.0
[R1, TRANSMISSION, 2021]: -1 AnnualTechnologyEmissionsPenalty[R1, TRANSMISSION, 2021] = -0.0
[R1, TRANSMISSION, 2022]: -1 AnnualTechnologyEmissionsPenalty[R1, TRANSMISSION, 2022] = -0.0
[R1, TRANSMISSION, 2023]: -1 AnnualTechnologyEmissionsPenalty[R1, TRANSMISSION, 2023] = -0.0
[R1, TRANSMISSION, 2024]: -1 AnnualTechnologyEmissionsPenalty[R1, TRANSMISSION, 2024] = -0.0
[R1, TRANSMISSION, 2025]: -1 AnnualTechnologyEmissionsPenalty[R1, TRANSMISSION, 2025] = -0.0
[R1, TRANSMISSION, 2026]: -1 AnnualTechnologyEmissionsPenalty[R1, TRANSMISSION, 2026] = -0.0
		...
[R1, HYDRO, 2064]: -1 AnnualTechnologyEmissionsPenalty[R1, HYDRO, 2064]               = -0.0
[R1, HYDRO, 2065]: -1 AnnualTechnologyEmissionsPenalty[R1, HYDRO, 2065]             

```ampl
s.t. E5_DiscountedEmissionsPenaltyByTechnology{r in REGION, t in TECHNOLOGY, y in YEAR}:
	AnnualTechnologyEmissionsPenalty[r,t,y] / DiscountFactorMid[r,y]
	=
	DiscountedTechnologyEmissionsPenalty[r,t,y];
```

In [52]:
con = m['AnnualTechnologyEmissionsPenalty'] / discount_factor_mid - m['DiscountedTechnologyEmissionsPenalty'] == 0
m.add_constraints(con, name='E5_DiscountedEmissionsPenaltyByTechnology')

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `E5_DiscountedEmissionsPenaltyByTechnology` (REGION: 1, YEAR: 51, TECHNOLOGY: 3):
--------------------------------------------------------------------------------------------
[R1, 2020, TRANSMISSION]: +0.9759 AnnualTechnologyEmissionsPenalty[R1, TRANSMISSION, 2020] - 1 DiscountedTechnologyEmissionsPenalty[R1, TRANSMISSION, 2020]  = -0.0
[R1, 2020, MINE_WATER]: +0.9759 AnnualTechnologyEmissionsPenalty[R1, MINE_WATER, 2020] - 1 DiscountedTechnologyEmissionsPenalty[R1, MINE_WATER, 2020]        = -0.0
[R1, 2020, HYDRO]: +0.9759 AnnualTechnologyEmissionsPenalty[R1, HYDRO, 2020] - 1 DiscountedTechnologyEmissionsPenalty[R1, HYDRO, 2020]                       = -0.0
[R1, 2021, TRANSMISSION]: +0.9294 AnnualTechnologyEmissionsPenalty[R1, TRANSMISSION, 2021] - 1 DiscountedTechnologyEmissionsPenalty[R1, TRANSMISSION, 2021]  = -0.0
[R1, 2021, MINE_WATER]: +0.9294 AnnualTechnologyEmissionsPenalty[R1, MINE_WATER, 2021] - 1 DiscountedTechnologyEmissionsPenalty[R1, MINE_WATER, 2021]        =

```ampl
s.t. E6_EmissionsAccounting1{r in REGION, e in EMISSION, y in YEAR}:
	sum{t in TECHNOLOGY}
	AnnualTechnologyEmission[r,t,e,y]
	=
	AnnualEmissions[r,e,y];
```

In [53]:
if ds['EMISSION'].size > 0:
    con = m['AnnualTechnologyEmission'].sum(dims=['TECHNOLOGY']) - m['AnnualEmissions'] == 0
    m.add_constraints(con, name='E6_EmissionsAccounting1')

```ampl
s.t. E7_EmissionsAccounting2{r in REGION, e in EMISSION}:
	sum{y in YEAR} AnnualEmissions[r,e,y]
	=
	ModelPeriodEmissions[r,e] - ModelPeriodExogenousEmission[r,e];
```

In [54]:
if ds['EMISSION'].size > 0:
    con = m['ModelPeriodEmissions'] - m['AnnualEmissions'].sum('YEAR') == ds['ModelPeriodExogenousEmission'].fillna(0)
    m.add_constraints(con, name='E7_EmissionsAccounting2')

```ampl
s.t. E8_AnnualEmissionsLimit{r in REGION, e in EMISSION, y in YEAR: AnnualEmissionLimit[r, e, y] <> -1}:
	AnnualEmissions[r,e,y] + AnnualExogenousEmission[r,e,y]
	<=
	AnnualEmissionLimit[r,e,y];
```

In [55]:
if 'E8_AnnualEmissionsLimit' in m.constraints:
    m.remove_constraints('E8_AnnualEmissionsLimit')

In [56]:
con = m['AnnualEmissions'] <= ds['AnnualEmissionLimit'] - ds['AnnualExogenousEmission'].fillna(0)
mask = ds['AnnualEmissionLimit'] != -1
m.add_constraints(con, name='E8_AnnualEmissionsLimit', mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


Constraint `E8_AnnualEmissionsLimit` (REGION: 1, EMISSION: 0, YEAR: 51):
------------------------------------------------------------------------

```ampl
s.t. E9_ModelPeriodEmissionsLimit{r in REGION, e in EMISSION: ModelPeriodEmissionLimit[r, e] <> -1}:
	ModelPeriodEmissions[r,e]
	<=
	ModelPeriodEmissionLimit[r,e];
```

In [57]:
# con = m['ModelPeriodEmissions'] <= ds['ModelPeriodEmissionLimit']
mask = ds['ModelPeriodEmissionLimit'] != -1
m.add_constraints(lhs=m['ModelPeriodEmissions'], 
                  sign='<=', 
                  rhs=ds['ModelPeriodEmissionLimit'], 
                  name='E9_ModelPeriodEmissionsLimit', 
                  mask=mask)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataA

Constraint `E9_ModelPeriodEmissionsLimit` (REGION: 1, EMISSION: 0):
-------------------------------------------------------------------

# Objective Function
```ampl
minimize cost: sum{r in REGION, y in YEAR} TotalDiscountedCost[r,y];
```

In [58]:
objective = m['TotalDiscountedCost'].sum(dims=['REGION', 'YEAR'])
m.add_objective(expr=objective, overwrite=True)

/Users/adminuser/opt/anaconda3/envs/osemosys_linopy/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


# Solving

In [59]:
results_path = f"../results/{model_name}/"
if not os.path.exists(results_path):
    os.makedirs(results_path)

m.to_file(f'../results/{model_name}/{model_name}.lp')

Writing constraints.:  30%|███       | 18/60 [00:00<00:00, 172.69it/s]

Writing integer variables.: 100%|██████████| 1/1 [00:00<00:00, 764.97it/s]


PosixPath('../results/model_one/model_one.lp')

In [60]:
m

Linopy MILP model

Variables:
----------
 * RateOfDemand (REGION, TIMESLICE, FUEL, YEAR)
 * Demand (REGION, TIMESLICE, FUEL, YEAR)
 * NumberOfNewTechnologyUnits (REGION, TECHNOLOGY, YEAR)
 * NewCapacity (REGION, TECHNOLOGY, YEAR)
 * AccumulatedNewCapacity (REGION, TECHNOLOGY, YEAR)
 * TotalCapacityAnnual (REGION, TECHNOLOGY, YEAR)
 * RateOfActivity (REGION, TIMESLICE, TECHNOLOGY, MODE_OF_OPERATION, YEAR)
 * RateOfTotalActivity (REGION, TECHNOLOGY, TIMESLICE, YEAR)
 * TotalTechnologyAnnualActivity (REGION, TECHNOLOGY, YEAR)
 * TotalAnnualTechnologyActivityByMode (REGION, TECHNOLOGY, MODE_OF_OPERATION, YEAR)
 * TotalTechnologyModelPeriodActivity (REGION, TECHNOLOGY)
 * RateOfProductionByTechnologyByMode (REGION, TIMESLICE, TECHNOLOGY, MODE_OF_OPERATION, FUEL, YEAR)
 * RateOfProductionByTechnology (REGION, TIMESLICE, TECHNOLOGY, FUEL, YEAR)
 * ProductionByTechnology (REGION, TIMESLICE, TECHNOLOGY, FUEL, YEAR)
 * ProductionByTechnologyAnnual (REGION, TECHNOLOGY, FUEL, YEAR)
 * RateOfProduc

In [61]:
print(available_solvers)

['glpk', 'cbc']


In [64]:
m.solve(io_api='direct', log_fn=f'../results/{model_name}/gurobi.log')

IO setting 'direct' not available for glpk solver. Falling back to `lp`.
Writing integer variables.: 100%|██████████| 1/1 [00:00<00:00, 1011.89it/s]
Optimization failed: 
Status: warning
Termination condition: unknown
Solution: 0 primals, 0 duals
Objective: nan
Solver model: not available
Solver message: 



('warning', 'unknown')

In [63]:
results_path = f"../results/{model_name}/csv/"
if not os.path.exists(results_path):
    os.makedirs(results_path)
    
for variable, data in m.solution.items():
    data = data.to_dataframe(name=variable)
    data[data[variable] != 0.0].to_csv(f"../results/{model_name}/csv/{variable}.csv")